# Subthreshold Scaling

Broken notebook, needs fixing.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import itertools
from panqec.analysis import (
    get_results_df, get_thresholds_df, export_summary_tables,
    subthreshold_scaling
)
from panqec.plots._hashing_bound import plot_hashing_bound
# from panqec.plots._repthresh import plot_repthresh
from panqec.plots._threshold import (
    plot_deformedxps, plot_deformedzps, plot_crossing_collapse,
    plot_combined_threshold_vs_bias, plot_combined_triangles,
    plot_threshold_fss, plot_data_collapse, plot_crossing_example,
    plot_collapse_example, detailed_plot
)
import matplotlib.backends.backend_pdf
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from numpy.polynomial.polynomial import polyfit, Polynomial

from panqec.analysis import fit_subthreshold_scaling_cubic, get_subthreshold_fit_function
from panqec.utils import format_polynomial

from itertools import product

In [ ]:
date_stamp = pd.Timestamp.now().strftime('%Y-%m-%d')

# Change this to the run name.
run_name = 'sts_coprime_combined'

decoder_name = 'BPOSD'
output_dir = f'../temp/paper/share/{run_name}/results'
input_dir = f'../temp/paper/share/{run_name}/inputs'

# Change this to where you want your plots saved.
plots_dir = f'../temp/paper/plots/{date_stamp}_plots/'

In [ ]:
os.makedirs(plots_dir, exist_ok=True)
plot_pdf = os.path.join(plots_dir, '{}_{}.pdf'.format(
    pd.Timestamp.now().strftime('%Y-%m-%d'), os.path.basename(os.path.dirname(output_dir))
))
print(os.path.abspath(plot_pdf))

In [ ]:
job_list = [os.path.splitext(x)[0] for x in os.listdir(input_dir)]

In [ ]:
%pdb

In [ ]:
from glob import glob
import json

In [ ]:
results_file = glob(os.path.join(output_dir, '*', '*.json'))[-1]

In [ ]:
with open(results_file) as f:
    data = json.load(f)

x = np.arange(0, 1, 0.05)
y_se = 0.1 + 0.9*x
y = -10 + 10*x**2 + 2*y_se*np.random.randn(*x.shape)
w = 1/y_se

coefficients, _, _, _ = np.linalg.lstsq(
    np.vstack([w*np.ones_like(x), w*x, w*x**2]).T,
    y*w,
    rcond=None
)

x_interp = np.linspace(min(x), max(x), 101)
polyfit_coefficients = polyfit(x, y, deg=2, w=w)

plt.errorbar(x, y, yerr=y_se, fmt='o', capsize=2)
plt.plot(x_interp, np.vstack([np.ones_like(x_interp), x_interp, x_interp**2]).T.dot(coefficients))
polyfit_polynomial = Polynomial(polyfit_coefficients)
plt.plot(x_interp, polyfit_polynomial(x_interp), 'k--')
plt.show()

In [ ]:
results_df = get_results_df(job_list, output_dir=output_dir, input_dir=input_dir)
results_df = results_df.dropna(subset=['p_est'])
# Drop the 2x2x2 due to finite-size effects
results_df = results_df[results_df['size'] != (2, 2, 2)]
# thresholds_df, trunc_results_df, params_bs_list = get_thresholds_df(results_df)

In [ ]:
results_df['eta'] = results_df['label'].str.extract(r'bias-([\d.]+)-').astype(float)

pdf = matplotlib.backends.backend_pdf.PdfPages(plot_pdf)

for eta in np.sort(results_df['eta'].unique()):
    if pd.isna(eta):
        error_model_name = results_df[results_df['eta'].isna()]['error_model'].iloc[0]
    else:
        error_model_name = results_df[results_df['eta'] == eta]['error_model'].iloc[0]
    detailed_plot(plt, results_df, error_model=error_model_name, x_limits='auto', yscale=None, thresholds_df=thresholds_df)
    plt.suptitle(f'{decoder_name} {error_model_name}, $\\eta = {eta}$')
    pdf.savefig(plt.gcf())
    plt.show()

pdf.close()

$p' = A p^n$

$\log p' = \log A + n \log p$

$p' = Ae^{Bd^2}$

$p' = A p^{d^2}$

$\log p' = \log A + d^2 \ln p$

$\log p' = \log A + B d^2$

$\Delta \log(p') = \frac{d}{dp'} (\log p') \Delta p' = \frac{\Delta p'}{p'}$

In [ ]:
pdf = matplotlib.backends.backend_pdf.PdfPages(os.path.join(plots_dir, '{}_{}.pdf'.format(
    pd.Timestamp.now().strftime('%Y-%m-%d'), 'subthreshold_ansatz_fits'
)))

fig, ax = plt.subplots(2, 1, figsize=(6, 8))
for size in results_df['size'].unique():
    df_filt = results_df[
        (results_df['size'] == size)
        & (results_df['eta'].isna())
        & (results_df['probability'] <= 0.5)
    ].sort_values(by='probability')
    plt.sca(ax[0])
    plt.errorbar(df_filt['probability'], df_filt['p_est'], yerr=df_filt['p_se'], fmt='.-', capsize=5, label=size[0])
    plt.sca(ax[1])
    plt.errorbar(df_filt['probability'], df_filt['p_est'], yerr=df_filt['p_se'], fmt='.-', capsize=5, label=size[0])
plt.sca(ax[0])
plt.title(r'XZ4X coprime lattice $L\times(L + 1)\times (L + 1)$ at $\eta_Z=\infty$')
plt.legend(title='Size $L$')
plt.ylabel(r'Logical error rate $p_L$')
plt.xlabel(r'Physical error rate $p$')
plt.sca(ax[1])
plt.legend(title='Size $L$')
plt.ylabel(r'Logical error rate $p_L$')
plt.xlabel(r'Physical error rate $p$')
plt.yscale('log')
plt.ylim(1e-5, 1)

In [ ]:
chosen_probabilities = np.sort(
    results_df['probability'].unique()
)
sts_properties, gradient_coefficients = subthreshold_scaling(
    results_df, chosen_probabilities=chosen_probabilities
)

Make subthrshold scaling plots for many different kinds of ansatz.

In [ ]:
pdf = matplotlib.backends.backend_pdf.PdfPages(os.path.join(plots_dir, '{}_{}.pdf'.format(
    pd.Timestamp.now().strftime('%Y-%m-%d'), 'subthreshold_ansatz_fits'
)))
for ansatz, order in list(itertools.product(['simple', 'poly'], [0, 1, 2, 3, 4])) + [('free_power', 0)]:

    y_fit, y_data, fit_params = fit_subthreshold_scaling_cubic(results_df, order=order, ansatz=ansatz)

    print('p_L_th = {}'.format(np.exp(fit_params[0])))
    print('p_th = {}'.format(np.exp(fit_params[1])))

    if ansatz == 'simple':
        d_string = f'{fit_params[2]:.3f}L^{{{order}}}'
    elif ansatz == 'free_power':
        d_string = f'{fit_params[2]:.3f}L^{{{fit_params[3]:.3f}}}'
    else:
        d_string = format_polynomial('L', fit_params[2:])
    print('d =', d_string)

    default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    chosen_colors = (default_colors*int(
        np.ceil(len(chosen_probabilities)/len(default_colors))
    ))[:len(chosen_probabilities)]

    subthreshold_fit_function = get_subthreshold_fit_function(order=order, ansatz=ansatz)

    for color, probability in zip(chosen_colors, chosen_probabilities):
        df_filt = results_df[results_df['probability'] == probability]
        log_p_L = np.log(df_filt['p_est'])
        log_p = np.log(df_filt['probability'].values)
        L = df_filt['size'].apply(lambda x: min(x))
        L_interp = np.linspace(min(L), max(L), 101)
        log_p_L_fit = subthreshold_fit_function([np.ones_like(L_interp)*np.log(probability), L_interp], *fit_params)
        plt.plot(L_interp, np.exp(log_p_L_fit), color=color, linestyle='--')
        plt.errorbar(L, np.exp(log_p_L), yerr=df_filt['p_se'], fmt='o', capsize=5, label=str(np.round(probability, 5)), color=color)
    plt.yscale('log')
    plt.legend(title=r'Physical $p$')
    plt.ylabel(r'Logical Error Rate $p_L$')
    plt.title(
        f'Ansatz: $p_L \\approx p_{{L,th}}\\left(p/p_{{th}}\\right)^{{(d+1)/2}}$\n'
        f'$d\\approx {d_string}$\n'
        f'$p_{{th}}\\approx {np.exp(fit_params[1]):.3f}$, '
        f'$p_{{L,th}}\\approx {np.exp(fit_params[0]):.3f}$'
    )
    plt.xlabel('Distance $d$')
    pdf.savefig(plt.gcf(), bbox_inches='tight', pad_inches=0.1)
    plt.show()
pdf.close()

In [ ]:
pdf = matplotlib.backends.backend_pdf.PdfPages(os.path.join(plots_dir, '{}_{}.pdf'.format(
    pd.Timestamp.now().strftime('%Y-%m-%d'), 'subthreshold_scaling'
)))

sts_df = pd.DataFrame(sts_properties)
log_p_on_1_minus_p_interp = np.linspace(
    min(sts_df['log_p_on_1_minus_p']),
    max(sts_df['log_p_on_1_minus_p']),
    101
)

plt.figure(figsize=(8, 6))
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
chosen_colors = (default_colors*int(
    np.ceil(len(chosen_probabilities)/len(default_colors))
))[:len(chosen_probabilities)]

for color, props in zip(chosen_colors, sts_properties):
    d_interp = np.linspace(min(props['d']), max(props['d']), 101)
    plt.errorbar(
        props['d'], props['p_est'], yerr=props['p_se'],
        capsize=3, fmt='.',
        label=r'${:.2f}$'.format(props['probability']),
        color=color,
    )
    linear_polynomial = Polynomial(props['linear_coefficients'])
    quadratic_polynomial = Polynomial(props['quadratic_coefficients'])
    cubic_polynomial = Polynomial(props['cubic_coefficients'])
    plt.plot(
        d_interp,
        np.exp(linear_polynomial(d_interp)),
        color=color,
        linestyle='dashed'
    )
    plt.plot(
        d_interp,
        np.exp(quadratic_polynomial(d_interp)),
        color=color,
        linestyle='-'
    )
    plt.plot(
        d_interp,
        np.exp(cubic_polynomial(d_interp)),
        color=color,
        linestyle='dashdot'
    )
plt.xlabel(r'Code Distance $d$', fontsize=16)
plt.ylabel(r'Logical Failure Rate $\bar{p}$', fontsize=16)
plt.yscale('log')
# plt.title(r'Subthreshold scaling coprime $XZ^4X$ code at $\eta=%d$' % results_df['eta'].iloc[0])
plt.legend(loc='upper right', ncol=3, title='Physical Error Rate $p$')

inset = inset_axes(
    plt.gca(), width="30%",
    height='30%',
    loc=3
)

inset.plot(
    sts_df['log_p_on_1_minus_p'],
    sts_df['linear_fit_gradient'],
    'x'
)
gradient_polynomial = Polynomial(gradient_coefficients)
inset.plot(
    log_p_on_1_minus_p_interp,
    gradient_polynomial(log_p_on_1_minus_p_interp),
    '--',
    label='Slope {:.2f}'.format(gradient_coefficients[-1])
)
inset.yaxis.set_label_position("right")
inset.yaxis.tick_right()
inset.xaxis.set_label_position("top")
inset.xaxis.tick_top()
inset.set_xlabel(r'$\log[p/(1- p)]$')
inset.set_ylabel('Line Gradients')
inset.legend()

pdf.savefig(plt.gcf(), bbox_inches='tight', pad_inches=0.1)
plt.show()

pdf.close()